<a href="https://colab.research.google.com/github/javiercarpio57/03MIAR---Algoritmos-de-Optimizacion/blob/main/Practica_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from itertools import permutations
import numpy as np
import random

numbers = list(range(1, 10))
operations = ['+', '-', '*', '/']

In [2]:
numbers_combination = [i for i in permutations(numbers, 5)]
operations_combination = [i for i in permutations(operations)]
total = []

for n in numbers_combination:
  for x in operations_combination:
    total.append(f'{n[0]}{x[0]}{n[1]}{x[1]}{n[2]}{x[2]}{n[3]}{x[3]}{n[4]}')

##Fuerza Bruta

In [3]:
def fuerza_bruta(permutations):
  max = -np.inf
  operation_max = None

  min = np.inf
  operation_min = None
  for x in permutations:
    result = eval(x)

    if result > max:
      max = result
      operation_max = [x]
    elif result == max:
      operation_max = x

    if result < min:
      min = result
      operation_min = [x]
    elif result == min:
      operation_min = x
    
  return operation_max, max, operation_min, min

In [4]:
operation_max, max, operation_min, min = fuerza_bruta(total)

In [5]:
print(max, min)

78.83333333333333 -70.71428571428571


In [6]:
print(operation_max)

9*8+7-1/6


In [7]:
print(operation_min)

2/7-9*8+1


##Algoritmo genético

In [68]:
def expand_operation(operation):
  numbers = [operation[x] for x in range(len(operation)) if x % 2 == 0]
  operations = [operation[x] for x in range(len(operation)) if x % 2 != 0]
  return numbers, operations

def generate_solution(N):
  solutions = []
  for i in range(N):
    rand_num = random.randint(0, len(numbers_combination) - 1)
    rand_op = random.randint(0, len(operations_combination) - 1)

    num = numbers_combination[rand_num]
    op = operations_combination[rand_op]

    solutions.append(f'{num[0]}{op[0]}{num[1]}{op[1]}{num[2]}{op[2]}{num[3]}{op[3]}{num[4]}')

  return solutions

def crossover(population):
  children = []
  population_ = population.copy()
  while len(population_) > 1:
    father = population_.pop(random.randint(0, len(population_) - 1))
    mother = population_.pop(random.randint(0, len(population_) - 1))

    slice_point = random.randint(1, len(father) - 2)
    father_pt1, father_pt2 = father[:slice_point], father[slice_point:]
    mother_pt1, mother_pt2 = mother[:slice_point], mother[slice_point:]

    child1, child2 = father_pt1 + mother_pt2, mother_pt1 + father_pt2
    children.append(child1)
    children.append(child2)

  return children

def check_children(child):
  nums, ops = expand_operation(child)

  if len(nums) != len(list(set(nums))) or len(ops) != len(list(set(ops))):
    available_numbers = [i for i in range(1, 10) if str(i) not in nums]
    numbers_in_child = []
    for num in nums:
      if num not in numbers_in_child:
        numbers_in_child.append(num)
      else:
        new_num = available_numbers.pop(random.randint(0, len(available_numbers) - 1))
        numbers_in_child.append(str(new_num))
    
    available_ops = [i for i in ['+', '-', '*', '/'] if i not in ops]
    ops_in_child = []
    for op in ops:
      if op not in ops_in_child:
        ops_in_child.append(op)
      else:
        new_op = available_ops.pop(random.randint(0, len(available_ops) - 1))
        ops_in_child.append(str(new_op))
    
    corrected_child = f'{numbers_in_child[0]}{ops_in_child[0]}{numbers_in_child[1]}{ops_in_child[1]}{numbers_in_child[2]}{ops_in_child[2]}{numbers_in_child[3]}{ops_in_child[3]}{numbers_in_child[4]}'
    return corrected_child
  else:
    return child

def algoritmo_genetico(fitness, max_iter):
  porcentaje_inicial = 0.001
  poblacion_inicial = generate_solution(int(len(total) * porcentaje_inicial))

  resultado_fitness = list(map(lambda x: fitness(x), poblacion_inicial))
  
  poblacion_fitness = list(zip(poblacion_inicial, resultado_fitness))
  poblacion_fitness.sort(key = lambda x: x[1], reverse = True)
  
  cont = 0
  best_try = -np.inf
  tries = 0
  while cont < max_iter:
    seleccion = poblacion_fitness[:int(0.7*len(poblacion_fitness))]
    seleccion = [x[0] for x in seleccion]
    
    children = crossover(seleccion)
    children = [check_children(child) for child in children] + seleccion[:int(0.3*len(poblacion_fitness))]

    resultado_fitness = list(map(lambda x: fitness(x), children))
    poblacion_fitness = list(zip(children, resultado_fitness))
    poblacion_fitness.sort(key = lambda x: x[1], reverse = True)

    if poblacion_fitness[0][1] > best_try:
      best_try = poblacion_fitness[0][1]
      tries = 0
    else:
      tries += 1

    if tries > 0.25 * max_iter:
      break

    cont += 1

  print(f'Intentos para llegar al maximo:', cont)
  return poblacion_fitness[0]


In [69]:
fitness = lambda individuo: eval(individuo)

In [76]:
N = 100
children = algoritmo_genetico(fitness, N)

Intentos para llegar al maximo: 37


In [77]:
print(children)

('9*8+7-1/6', 78.83333333333333)
